In [1]:
%pylab inline
from numpywren.matrix import BigMatrix, BigSymmetricMatrix
from numpywren import matrix_utils, uops
from numpywren.matrix_init import shard_matrix
import pytest
import numpy as np
from numpy.linalg import cholesky
import pywren
import unittest
import concurrent.futures as fs
import time
import multiprocessing
import os
from importlib import reload
from numpywren import lambdapack as lp
from numpywren import binops
from collections import defaultdict
import seaborn as sns
import boto3
import numpywren
import numpywren.wait
from numpywren import job_runner
import pylab
import seaborn as sns
import math
sns.set_style("whitegrid")

Populating the interactive namespace from numpy and matplotlib


In [2]:
env_dict = {}
env_dict["OMP_NUM_THREADS"] = str(1)
env_dict["AWS_ACCESS_KEY_ID"] = os.environ["AWS_ACCESS_KEY_ID"]
env_dict["AWS_SECRET_ACCESS_KEY"] = os.environ["AWS_SECRET_ACCESS_KEY"]

In [3]:
N = 65536 * 4
shard_size = 5300 

#N = 1024
#shard_size = 64 

X = np.random.randn(N, 1)
print("Generating X")
shard_sizes = (shard_size, 1)
pwex = pywren.default_executor()
executor = pywren.default_executor


Generating X


In [4]:
X_sharded = BigMatrix("cholesky_test_{0}_{1}".format(N, shard_size), shape=X.shape, shard_sizes=shard_sizes, write_header=True)


In [5]:
shard_matrix(X_sharded, X)

In [6]:
%time XXT_sharded = binops.gemm(pwex, X_sharded, X_sharded.T, overwrite=False)

gemm(BigMatrix(cholesky_test_262144_5300), BigMatrix(cholesky_test_262144_5300).T)
Out Shape (262144, 262144)
Total number of output blocks 1275
Total number of output blocks that exist 1275
Number of output blocks to generate  0
<function _gemm_remote_0 at 0x7f13d829b400>
CPU times: user 412 ms, sys: 0 ns, total: 412 ms
Wall time: 1.01 s


In [7]:
XXT_sharded.lambdav = 1
%time instructions_full,L_sharded,trailing= lp._chol(XXT_sharded)

CPU times: user 924 ms, sys: 16 ms, total: 940 ms
Wall time: 1.01 s


In [8]:
for i,inst_block in enumerate(instructions_full):
   for inst in inst_block.instrs:
       if (inst.i_code == lp.OC.CHOL):
           print(i)


0
1275
2500
3676
4804
5885
6920
7910
8856
9759
10620
11440
12220
12961
13664
14330
14960
15555
16116
16644
17140
17605
18040
18446
18824
19175
19500
19800
20076
20329
20560
20770
20960
21131
21284
21420
21540
21645
21736
21814
21880
21935
21980
22016
22044
22065
22080
22090
22096
22099


In [9]:
len(instructions_full)

22100

In [10]:
INSTRUCTION_TO_TRUNCATE = 5000
NUM_CORES = 100
NUM_MAX_CORES = 3000

In [11]:
instructions = instructions_full[:INSTRUCTION_TO_TRUNCATE]


In [12]:
pwex = pywren.default_executor()
config = pwex.config

In [13]:
program = lp.LambdaPackProgram(instructions, executor=pywren.lambda_executor, pywren_config=config, num_priorities=6)
print(program.longest_path)


serializing program before sharding
sharding program
Sharding program took 13.94001579284668 seconds
[0, 1, 50, 1275, 1276, 1324, 2500, 2501, 2548, 3676, 3677, 3723, 4804, 4805, 4850, 5000]


In [14]:
#len(program.inst_blocks)
pwex = pywren.lambda_executor()

In [15]:
program.start()

0

In [16]:
print(program.hash)

bbdf6c210b963763877b81d10b0ead3b2e6d0a47


In [17]:
# all_futures = pwex.map(lambda x: job_runner.lambdapack_run(program, pipeline_width=1, cache_size=2, timeout=100), 
#                        range(NUM_CORES), 
#                        exclude_modules=["site-packages"], 
#                        extra_env={"REDIS_IP":os.environ["REDIS_IP"]})
all_futures = []
all_future_set = []
running_total = []
waiting_total = []
up_total = []
measure_points = []

current_task_id = 0
client = boto3.client('sqs')

def launch_more_tasks(num_launch):
        assert num_launch > 0
        global current_task_id
        global client
        global program
        
        for i in range(current_task_id,current_task_id+num_launch):
            client.send_message(QueueUrl=program.worker_queue_url, MessageBody=str(i))
        handles = []
        while len(handles) != num_launch:
            messages = client.receive_message(QueueUrl=program.worker_queue_url, MaxNumberOfMessages=min(10, num_launch-len(handles)))
            if "Messages" not in messages:
                continue
            for m in messages["Messages"]:
                handles.append(m['ReceiptHandle'])
        
        more_futures = pwex.map(lambda x: job_runner.lambdapack_run(program, x, pipeline_width=2, cache_size=0, timeout=100, idle_timeout=10), 
                   handles, 
                   exclude_modules=["site-packages"], 
                   extra_env={"REDIS_IP":os.environ["REDIS_IP"], "REDIS_PASS": "numpywren123", "REDIS_PORT":"6379"})
        current_task_id += num_launch
        program.incr_pool_size(num_launch)
        return more_futures


In [ ]:
five_point_vector = [False]*5
five_point_pos = 0
consecutive_count = 0
tzero = time.time()
scaling_factor = 0.5

last_vector = []
no_progress = 0
while(program.program_status() == lp.PS.RUNNING):
    time.sleep(1)
    waiting = 0
    running = 0
    client = boto3.client('sqs')
    for p,queue_url in enumerate(program.queue_urls):
        #print("Priority {0}".format(p))
        attrs = client.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['ApproximateNumberOfMessages', 'ApproximateNumberOfMessagesNotVisible'])['Attributes']
        #print(attrs)
        waiting += int(attrs["ApproximateNumberOfMessages"])
        running += int(attrs["ApproximateNumberOfMessagesNotVisible"])
        # Implement scale up policy here
    attrs = client.get_queue_attributes(QueueUrl=program.worker_queue_url, AttributeNames=['ApproximateNumberOfMessages', 'ApproximateNumberOfMessagesNotVisible'])['Attributes']
    w_waiting = int(attrs["ApproximateNumberOfMessages"])
    w_running = int(attrs["ApproximateNumberOfMessagesNotVisible"])

    five_point_pos = (five_point_pos + 1) % len(five_point_vector)
    five_point_vector[five_point_pos] = waiting > 0
    up = int(program.get_pool_size())
    w_up = w_running
    #if (waiting > 0 and up < waiting + running and up < NUM_MAX_CORES):
    #    all_futures += launch_more_tasks(min(waiting, waiting+running-up, NUM_MAX_CORES-up))
    #if w_up != 1:
    #    all_future_set.append(launch_more_tasks(1))
    #if up != 1:
    #    all_future_set.append(launch_more_tasks(1))
    #if (up < math.ceil(1.0*(waiting)*scaling_factor) and up < NUM_MAX_CORES):
    #    all_future_set.append(launch_more_tasks(min(int(math.ceil(1.0*(waiting)*scaling_factor-up)), NUM_MAX_CORES-up)))
    if (w_up < math.ceil(1.0*(waiting)*scaling_factor) and up < NUM_MAX_CORES):
        all_future_set.append(launch_more_tasks(min(int(math.ceil(1.0*(waiting)*scaling_factor-w_up)), NUM_MAX_CORES-w_up)))
    
#     if five_point_pos == (len(five_point_vector) - 1):
#         if all(five_point_vector):
#             all_futures += launch_more_tasks(2**consecutive_count)
#             consecutive_count += 1
#         else:
#             consecutive_count = 0
    waiting_total.append(waiting)
    running_total.append(running)
    up_total.append(up)
    measure_points.append(time.time() - tzero)
    if last_vector == [waiting, running, up]:
        no_progress += 1
        if no_progress == 30:
            program.decr_pool_size(up)
            print("resetting counter.")
    else:
        last_vector = [waiting, running, up]
        no_progress = 0
    if len(waiting_total) % 10 == 0:
        output_vec = [time.time() - tzero, waiting, running, up, " counter: ", program.get_max_pc(), "--", w_waiting, w_running]
        output_str = [str(item) for item in output_vec]
        print(output_str)

['12.721174001693726', '0', '1', '2', ' counter: ', '0', '--', '0', '2']
['24.839184284210205', '47', '2', '24', ' counter: ', '2', '--', '0', '24']
['37.13138723373413', '15', '34', '47', ' counter: ', '45', '--', '0', '47']
['47.8746178150177', '0', '50', '47', ' counter: ', '52', '--', '0', '47']
['58.76234698295593', '10', '57', '30', ' counter: ', '679', '--', '0', '30']
['79.90936946868896', '697', '94', '534', ' counter: ', '1084', '--', '0', '534']
['90.83994507789612', '831', '298', '534', ' counter: ', '1274', '--', '0', '534']
['102.6289451122284', '134', '1064', '534', ' counter: ', '1275', '--', '2', '532']


In [ ]:
# waiting = 0
# running = 0
# for queue_url in program.queue_urls[::-1]:
#     attrs = client.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['ApproximateNumberOfMessages', 'ApproximateNumberOfMessagesNotVisible'])['Attributes']
#     #print(attrs)
#     waiting += int(attrs["ApproximateNumberOfMessages"])
#     running += int(attrs["ApproximateNumberOfMessagesNotVisible"])
#     messages = client.receive_message(QueueUrl=queue_url, MaxNumberOfMessages=1)
#     if ("Messages" not in messages):
#         break
#         #continue
#     else:
#         break
# if ("Messages" not in messages):
#     print("no msg: " + str(waiting) + " " + str(running))
# else:
#     print("msg: " + str(waiting) + " " + str(running))


In [ ]:
program.free()

In [ ]:
cost = trapz(up_total, measure_points)
duration = max(measure_points)
print("duration: " + str(duration) + " cost: " + str(cost))

In [ ]:
all_futures = [f for fs in all_future_set for f in fs]

In [ ]:
max(measure_points) / 60

In [ ]:
#[f.result()['up_time'][1] - f.result()['up_time'][0] for f in all_futures]

In [ ]:
#f = all_futures[0]
#f.run_status

In [ ]:
print(program.program_status())

In [ ]:
print(program.hash)

In [ ]:
executor = fs.ThreadPoolExecutor(64)
futures = []
for i in range(0,len(program.inst_blocks),1):
    
    futures.append(executor.submit(program.get_profiling_info, i))
    
    

In [ ]:
print(len(futures))


In [ ]:
%time res = fs.wait(futures)



In [ ]:
profiled_blocks = []
for f in futures:
    try:
        profiled_blocks.append(f.result())
    except:
        pass
    

In [ ]:
from matplotlib import patches as mpatches
time_offset = np.min([i.start_time for b in profiled_blocks for i in b.instrs if i.start_time != None and i.end_time != None])
instructions = [i for b in profiled_blocks for i in b.instrs if i.start_time != None and i.end_time != None]
fig = pylab.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
total_jobs = len(instructions)
y = np.arange(len(instructions))
point_size = 50

start_times = np.array([i.start_time for i in instructions]) - time_offset
end_times = np.array([i.end_time for i in instructions]) - time_offset
palette = sns.color_palette("deep", 6)
patches = []

READ_INSTRUCTIONS = [lp.OC.S3_LOAD]
WRITE_INSTRUCTIONS = [lp.OC.S3_WRITE]
COMPUTE_INSTRUCTIONS = [lp.OC.SYRK, lp.OC.TRSM, lp.OC.INVRS, lp.OC.CHOL]

read_starts = []
read_ends = []
write_starts = []
write_ends = []
compute_starts = []
compute_ends = []
read_ys = []
write_ys = []
compute_ys = []
write_times = []
compute_times = []
read_times = []
for j,block in enumerate(profiled_blocks):
    syrk = False
    for i in block.instrs:
        if i.start_time == None or i.end_time == None: 
            continue
        if (i.i_code in READ_INSTRUCTIONS):
            read_starts.append(i.start_time - time_offset)
            read_ends.append(i.end_time - time_offset)
            read_times.append(i.end_time - i.start_time)
            read_ys.append(j) 
            
        elif (i.i_code in COMPUTE_INSTRUCTIONS):
            if (i.i_code == lp.OC.SYRK):
                syrk = True
                compute_times.append(i.end_time - i.start_time)
            compute_starts.append(i.start_time - time_offset)
            compute_ends.append(i.end_time - time_offset)
            compute_ys.append(j)
        elif (i.i_code in WRITE_INSTRUCTIONS):
            
            if (syrk):
                write_starts.append(i.start_time - time_offset)
                write_ends.append(i.end_time - time_offset)
                write_times.append(write_ends[-1] - write_starts[-1])
                write_ys.append(j)
       

for i,instr in enumerate(instructions):
    if (instr.i_code == lp.OC.CHOL):
        ax.axvline(instructions[i+1].start_time - time_offset, color="black", linestyle='--')
        ax.axvline(instructions[i+1].end_time - time_offset, color="black")
        
        
        
ax.scatter(read_starts, read_ys,  c=palette[0], edgecolor='none', s=point_size, alpha=0.8)
patches.append(mpatches.Patch(color=palette[0], label="read_argument"))
ax.scatter(read_ends, read_ys,  c=palette[1], edgecolor='none', s=point_size, alpha=0.8)
patches.append(mpatches.Patch(color=palette[1], label="read_end"))

ax.scatter(compute_starts, compute_ys,  c=palette[2], edgecolor='none', s=point_size, alpha=0.8)
patches.append(mpatches.Patch(color=palette[2], label="compute_start"))
ax.scatter(compute_ends, compute_ys,  c=palette[3], edgecolor='none', s=point_size, alpha=0.8)
patches.append(mpatches.Patch(color=palette[3], label="compute_end"))

ax.scatter(write_starts, write_ys,  c=palette[4], edgecolor='none', s=point_size, alpha=0.2)
patches.append(mpatches.Patch(color=palette[4], label="write_start"))
ax.scatter(write_ends, write_ys,  c=palette[5], edgecolor='none', s=point_size, alpha=0.2)
patches.append(mpatches.Patch(color=palette[5], label="write_result"))
ax.set_xlabel('wallclock time (sec)')
ax.set_ylabel('job')

legend = pylab.legend(handles=patches, 
                      loc='upper left', frameon=True)
#pylab.title("Runtime for {} jobs of {:3.0f}M double ops (dgemm) each".format(total_jobs, JOB_GFLOPS))
legend.get_frame().set_facecolor('#FFFFFF')
#ax.set_ylim(2500,2510)
#ax.set_xticks(np.arange(20))
#ax.set_xlim(70,80)

In [ ]:
plt.hist(read_times, bins=100)

In [ ]:
plt.hist(compute_times, bins=100)
plt.xlabel("Low Rank Update Time")
plt.ylabel("Number of jobs completed")

In [ ]:
read,write,total_flops,bins, instructions, runtimes = lp.perf_profile(profiled_blocks, num_bins=100)


In [ ]:
optimes = defaultdict(int)
opcounts = defaultdict(int)

In [ ]:
for inst, t in zip(instructions, runtimes):
    opcounts[inst.i_code] += 1
    optimes[inst.i_code] += t


In [ ]:
for k in optimes.keys():
    print("{0}: {1}s".format(k, optimes[k]/opcounts[k]))
    

In [ ]:
print(profiled_blocks[32].end_time - profiled_blocks[0].start_time)

In [ ]:
for k in optimes.keys():
    print("{0}: {1}s".format(k, optimes[k]/opcounts[k]))
    

In [ ]:
sns.tsplot(read, time=(bins - min(bins)))
plt.xlabel("time since start")
plt.ylabel("aggregate read GB/s")
#plt.xlim(10,12)


In [ ]:
#print(bins)

In [ ]:
sns.tsplot(write, time=(bins - min(bins)))
plt.xlabel("time since start")
plt.ylabel("aggregate write GB/s")
#plt.xlim(10,12)


In [ ]:
pow(3*sum(total_flops), 1.0/3.0)

In [ ]:
c = sns.palettes.color_palette()[1]
sns.tsplot(total_flops, time=(bins - min(bins)), condition="Observed Performance")
plt.xlabel("Time since start")
plt.ylabel("aggregate Gflops/s")
plt.hlines(20*NUM_CORES,0,1000, label="Theoretical Performance", color=c)
plt.legend(bbox_to_anchor=(1.5, 0.8))
#plt.savefig("flops_100k.pdf")
#plt.xlim(9,10)

In [ ]:
((16384**3)/3)/(1e9)

In [ ]:
len(all_futures)

In [ ]:
# this does not work in current PyWren as it has multiple callsets
# pywren.wait(all_futures)

In [ ]:
import concurrent.futures as fss
import redis
#import pywren.storage.StorageOutputNotFoundError

all_results = []
bad_counts = 0
bad_counts_1 = 0
bad_counts_2 = 0
bad_counts_3 = 0
bad_future = None
count = 0
for fs in all_future_set[:20]:
    #print(len(all_results))
    try:
        pywren.wait(fs)
    except Exception as e:
        print("wait error")
    for f in fs:
        count += 1
        #print(count)
        try:
           result = f.result()
           all_results.append(result)
        except fss._base.TimeoutError as e:
            print(e)
            bad_counts += 1
            continue
        except redis.connection.ResponseError as e:
            #print(e)
            bad_counts_1 += 1
            continue
        except Exception as e:
            if str(e) == "process ran out of time":
                bad_counts_2 += 1
            else:
                bad_counts_3 += 1
                print(e)
            continue
print("bad counts: " + str(bad_counts))
print("bad counts_1: " + str(bad_counts_1))
print("bad counts_2: " + str(bad_counts_2))
print("bad counts_3: " + str(bad_counts_3))
# all_results = [f.result() for f in all_futures]

In [ ]:
len(all_future_set)

In [ ]:
exec_times = [exec_time for res in all_results for exec_time in res['exec_time']]
tasks_per_worker = [len(res['exec_time']) for res in all_results]
up_times = [res['up_time'] for res in all_results]
#print((up_times))

print(len(exec_times))
#gap_times = [res['up_time'][1] - max([max(t) for t in res['exec_time']] + [res['up_time'][1]]) for res in all_results]
up_duration = [up[1] - up[0] for up in up_times]
exec_duration = [up[1] - up[0] for up in exec_times]
plt.hist(up_duration, bins=100)
plt.xlabel("task execution time (seconds)")
plt.ylabel("counts (seconds)")
plt.show()
# print(up_times)

In [ ]:
all_results[1]

In [ ]:
runtime_bins = np.linspace(0, int(max(measure_points) + 1), int(max(measure_points) + 1) * 2)

def compute_times_rates(d, tzero):
    
    x = np.array(d)
    # tzero = np.min(x[:, 0])
    start_time = x[:, 0] - tzero
    end_time = x[:, 1]  - tzero

    N = len(start_time)

    runtime_jobs_hist = np.zeros((N, len(runtime_bins)))

    for i in range(N):
        s = start_time[i]
        e = end_time[i]
        a, b = np.searchsorted(runtime_bins, [s, e])
        if b-a > 0:
            runtime_jobs_hist[i, a:b] = 1

    return {'start_time' : start_time, 
            'end_time' : end_time, 
            'runtime_jobs_hist' : runtime_jobs_hist}

exec_timeline = compute_times_rates(exec_times, tzero)
up_timeline = compute_times_rates(up_times, tzero)

from matplotlib.collections import LineCollection

    
fig = pylab.figure(figsize=(8, 4))

current_palette = sns.color_palette()
up_color = current_palette[0]
exec_color = current_palette[1]

ax = fig.add_subplot(1, 1, 1)

for plot_i, (datum, l, c) in enumerate([(exec_timeline, 'busy workers', up_color), 
                                    (up_timeline, 'ready workers', exec_color)]):

    ax.plot(runtime_bins, datum['runtime_jobs_hist'].sum(axis=0), 
            c=c, label=l, 
           zorder=-1)


    ax.set_xlim(0, np.max(datum['end_time']))
    ax.set_xlabel("time (sec)")

#ax.plot(measure_points, running_total, 'r', label="running tasks")
#ax.plot(measure_points, waiting_total, 'g', label="tasks in queue")
ax.plot(measure_points, up_total, 'black', label="up workers")

ax.set_ylim(0, max(up_total)*1.05)
ax.set_ylabel("num workers")
ax.legend(loc='upper right')
fig.tight_layout()


In [ ]:
fig = pylab.figure(figsize=(8, 4))

ax = fig.add_subplot(1, 1, 1)
ax.plot(measure_points, running_total, 'r', label="running tasks")
ax.plot(measure_points, waiting_total, 'g', label="tasks in queue")
ax.plot(measure_points, up_total, 'black', label="up counter")
ax.set_ylabel("num workers")
ax.legend(loc='upper right')
fig.tight_layout()



In [ ]:
busy_area = trapz(exec_timeline['runtime_jobs_hist'].sum(axis=0), runtime_bins)
ready_area = trapz(up_timeline['runtime_jobs_hist'].sum(axis=0), runtime_bins)
up_area = trapz(up_total, measure_points)
print({"effec_util":busy_area/ready_area,
       "startup_overhead": 1-ready_area/up_area,
       "overall_util":busy_area/up_area})

In [ ]:
len(runtime_bins)

In [ ]:
import pickle
results = {"runtime_bins": runtime_bins,
          "exec_timeline": exec_timeline,
          "up_timeline": up_timeline,
          "measure_points": measure_points,
          "running_total": running_total,
          "waiting_total": waiting_total,
          "up_total": up_total}
pickle.dump(results, open('cholesky_256k_moderate_10s.pickle', 'wb'))


In [ ]:
results = pickle.load(open('cholesky_256k_moderate_10s.pickle', 'rb'))
up_total = results["up_total"]


In [ ]:
fig = pylab.figure(figsize=(8, 4))

ax = fig.add_subplot(1, 1, 1)

c = sns.palettes.color_palette()
sns.tsplot(total_flops, time=(bins - min(bins)), condition="Observed Performance", color=c[1])
sns.tsplot(np.multiply(up_total, 20), time=measure_points, condition="Theoretical Peak Performance", color=c[2])
sns.tsplot(np.multiply(exec_timeline['runtime_jobs_hist'].sum(axis=0), 20), time=runtime_bins, condition="Theoretical Running Performance", color=c[3])
#sns.tsplot(np.multiply(up_timeline['runtime_jobs_hist'].sum(axis=0), 20), time=runtime_bins, condition="Theoretical Available Performance", color=c[4])

plt.xlabel("Time since start")
plt.ylabel("aggregate Gflops/s")
# plt.hlines(20*NUM_CORES,0,1000, label="Theoretical Performance", color=c)
plt.legend(bbox_to_anchor=(1.5, 0.8))
plt.savefig("flops_autoscaling.pdf")
#plt.xlim(9,10)



In [ ]:
trapz(total_flops, bins) / busy_area / 20

In [ ]:
cpu_type_count = {}
for f in all_futures:
    if f.done():
        model = f.run_status['server_info']['/proc/cpuinfo'].split("\n")[4]
        if model not in cpu_type_count:
            print (f.run_status['server_info']['/proc/cpuinfo'].split("\n"))
            cpu_type_count[model] = 0
        cpu_type_count[model] += 1
print(cpu_type_count)

32
18